In [1]:
import asyncio, json, uuid, re, httpx, sseclient

BASE = "http://localhost:8000"

class LexMCP:
    def __init__(self, base: str = BASE):
        self.base = base
        self.session_id = None
        self.messages_url = None
        self._stream = None
        self._pending = {}             # id -> Future

    async def __aenter__(self):
        # 1) open the SSE stream once
        client = httpx.AsyncClient(timeout=60.0)
        resp = await client.get(f"{self.base}/mcp", timeout=None)
        self._stream = sseclient.SSEClient(resp)
        # first data: line gives us /mcp/messages/?session_id=...
        first = await asyncio.get_event_loop().run_in_executor(None, self._stream.__next__)
        path  = first.data.strip()
        self.messages_url = f"{self.base}{path}"
        m = re.search(r"session_id=([a-f0-9]+)", path)
        self.session_id = m.group(1)
        # background task to read events
        asyncio.create_task(self._listen())
        self._client = client
        return self

    async def __aexit__(self, *exc):
        await self._client.aclose()

    async def _listen(self):
        for event in self._stream:
            try:
                obj = json.loads(event.data)
                if obj.get("jsonrpc") != "2.0":
                    continue
                req_id = obj.get("id")
                if req_id in self._pending:
                    self._pending[req_id].set_result(obj.get("result"))
            except Exception:
                continue

    async def call(self, method: str, params: dict):
        req_id = str(uuid.uuid4())
        payload = {
            "jsonrpc": "2.0",
            "id": req_id,
            "method": method,
            "params": params,
        }
        fut = asyncio.get_event_loop().create_future()
        self._pending[req_id] = fut
        # 2) post the request (fire‑and‑forget)
        await self._client.post(self.messages_url, json=payload)
        # 3) await the response pushed on the stream
        result = await fut
        del self._pending[req_id]
        return result


In [3]:
import asyncio

async def demo():
    async with LexMCP() as mcp:
        hits = await mcp.call("search_legislation_section",
                              {"query": "climate change", "size": 2})
        print(hits)

# asyncio.run(demo())
await demo()


CancelledError: 

In [ ]:
# ONE‑CELL agent: correct "name"+"arguments" payload  ➜ Azure OpenAI
# ------------------------------------------------------------------
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, uuid, re, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
BASE  = "http://localhost:8000"
MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_KEY"),
    api_version    = "2025-04-01-preview",
)

def lex_session_url(base=BASE):
    r = requests.get(f"{base}/mcp", stream=True, timeout=30)
    for ln in r.iter_lines(decode_unicode=True):
        if ln.startswith("data:"):
            return f"{base}{ln[5:].strip()}"

MCP_URL = lex_session_url()
print("Lex session →", MCP_URL)

def call_lex(tool: str, args: dict):
    body = {"name": tool, "arguments": args}          # ← name + arguments
    msg  = {
        "id": str(uuid.uuid4()),
        "role": "user",
        "content": json.dumps(body),                  # ← stringify
    }
    r = requests.post(MCP_URL, json=[msg], timeout=30)
    if r.status_code != 202:
        print("SERVER:", r.status_code, r.text[:200]) # debug
    r.raise_for_status()
    return r.json()["results"]

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_legislation_section",
            "description": "Search UK legislation sections.",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"},
                               "size":  {"type": "integer", "default": 3}},
                "required": ["query"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_caselaw_section",
            "description": "Search judgments that cite a section.",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"},
                               "size":  {"type": "integer", "default": 3}},
                "required": ["query"]
            },
        },
    },
]

def chat(q: str) -> str:
    msgs = [
        {"role": "system",
         "content": "You are a UK public‑law assistant. Use the tools when needed."},
        {"role": "user", "content": q},
    ]
    first = oai.chat.completions.create(
        model=MODEL, messages=msgs, tools=TOOLS, tool_choice="auto"
    ).choices[0].message
    msgs.append(first)

    if first.tool_calls:
        for call in first.tool_calls:
            args = json.loads(call.function.arguments)
            hits = call_lex(call.function.name, args)
            msgs.append({"role":"tool",
                         "tool_call_id": call.id,
                         "name": call.function.name,
                         "content": json.dumps(hits)})

    final = oai.chat.completions.create(model=MODEL, messages=msgs)
    return final.choices[0].message.content

print(chat("Summarise the duty in section 5 of the Environment Act 2021 and cite any cases interpreting it."))


/Users/radoslawkowalczyk/lex/.venv/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Lex session URL  → http://localhost:8000/mcp/messages/?session_id=79bb44077ae847e28f3f0fdcf5278e44
STATUS 202


KeyError: 'results'